#Riassunto e Identificazione delle Attività dalla Trascrizione di un Meeting

In [11]:
"""
Installazione librerie, gruppo langchain per gestire i prompt e input-output e così via
La scelta della libreria pdfplumber è ricaduta sul fatto che oltre a leggere bene il testo dei pdf, si comporta molto bene con pandas
e con le tabelle e immagini per i dati così che sia più completo
"""
!pip install langchain langchain-community langchain_groq
!pip install pdfplumber

## Creazione di una funzione che pulisce il testo da stopword e numeri e poi lo tokenizza per migliorare l'elaborazione dei dati.

In [12]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('punkt_tab')
nltk.download('stopwords')

def clean_and_tokenize_text(text):
    """
    Pulisce un testo rimuovendo numeri, stopwords italiane e restituisce i token.
    Args:
        text (str): Il testo da pulire.
    Returns:
        list: Lista di token puliti.
    """
    # Rimuove i numeri
    text_no_numbers = re.sub(r'\d+', '', text)

    # Tokenizza il testo
    words = word_tokenize(text_no_numbers, language='italian')

    # Carica le stopwords italiane
    stop_words = set(stopwords.words('italian'))

    # Filtra: solo parole alfabetiche e non stopwords
    transcription_text = [word.lower() for word in words if word.isalpha() and word.lower() not in stop_words]

    return transcription_text

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
#Altra funzione

def clean_and_tokenize_text(text):
    """
    Pulisce un testo rimuovendo numeri, stopwords italiane e restituisce i token.
    Args:
        text (str): Il testo da pulire.
    Returns:
        list: Lista di token puliti.
    """
    text_no_numbers = re.sub(r'\d+', '', text)
    words = word_tokenize(text_no_numbers, language='italian')
    stop_words = set(stopwords.words('italian'))
    cleaned_tokens = [word.lower() for word in words if word.isalpha() and word.lower() not in stop_words]
    return cleaned_tokens

#Try con una trascrizione recuperata da 'url'

###In questo test con il modello, andremo a prendere la trascrizione del meeting si fa la pulizia del testo e la tokenizzazione, ho cercato di intercettare i vari casi di immissione della tipologia del file, se vengono messi entrambi o nessuno.
### Una volta elaborata e pulita la trascrizione si procede con la trascrizione del modello, e successivamente la preparazione del prompt, dando il contesto all'LLM e il formato desiderato in output.
###Infine si crea la catena che collega modello, prompt e template(che in questo caso sono già uniti) e si va a runnare il contenuto.

In [14]:
import requests
import pdfplumber
import nltk
from langchain_groq import ChatGroq
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate


url = "https://raw.githubusercontent.com/Profession-AI/progetti-llm/refs/heads/main/Riassunto%20e%20identificazione%20delle%20attivit%C3%A0%20dalla%20trascrizione%20di%20un%20meeting/meeting_transcription.txt"
pdf = None

transcription_text = ""

#Caricamento testo
try:
    if url and not pdf:
        response = requests.get(url)
        response.raise_for_status()
        transcription_text = response.text.strip()
    elif pdf and not url:
        with pdfplumber.open(pdf) as pdf_file:
            transcription_text = "\n".join(
                page.extract_text() for page in pdf_file.pages if page.extract_text()
            ).strip()
        if not transcription_text:
            raise ValueError("Il pdf non ha testo leggibile all'interno")
    else:
        raise ValueError("Per favore inserisci solo una tra le due opzioni: url o pdf")
except Exception as e:
    print(f"Errore durante il caricamento: {e}")
    transcription_text = ""

#Se si ha il testo
if transcription_text:
    # Pulizia e tokenizzazione
    cleaned_tokens = clean_and_tokenize_text(transcription_text)
    cleaned_text = ' '.join(cleaned_tokens)

    # Configura il modello LLaMA tramite Groq
    llm = ChatGroq(
        api_key='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',
        model='llama3-70b-8192'
    )

    # Prepara il prompt
    prompt_goal = PromptTemplate(
        input_variables=["transcription"],
        template="""
Sei un assistente per uffici esperto. Leggi la seguente trascrizione di un meeting e produci:
- Un riassunto diviso nei punti chiave discussi durante il meeting
- Per ogni persona del meeting crea un elenco dove ci sono i compiti che deve svolgere o supervisionare.


Trascrizione:
{transcription}

Rispondi SOLO nel formato:
---
Riassunto:
- Punto 1
- Punto 2
...

Attività:
- Persona A: attività
- Persona B: attività
...
---
"""
    )

    # Crea la catena LLM
    output = LLMChain(llm=llm, prompt=prompt_goal)

    # Esegui la catena
    try:
        result = output.run({"transcription": cleaned_text})
        if "---" in result:
            blocks = result.split('---')
            clean_output = blocks[1].strip() if len(blocks) > 1 else result
            print(clean_output)
        else:
            print(result)
    except Exception as e:
        print(f"Errore durante l'elaborazione dell'output: {e}")
else:
    print("Nessuna trascrizione valida disponibile per l'elaborazione.")


Riassunto:
- Requisiti del nuovo sistema contabile: intuitività, gestione automatizzata, reportistica personalizzabile, modulo per filtri e grafici, automazione scadenze con notifiche promemoria, archiviazione digitale di documenti con funzione di ricerca avanzata.
- Richiesta di creazione di un piano tecnico basato sui requisiti emersi e approvazione del progetto definitivo.

Attività:
- Rossana Bolletta: nulla (ha già espresso le esigenze)
- Mario Rossi: supervisionare il progetto e approvare il piano tecnico
- Andrea Monti: creare un piano tecnico basato sui requisiti emersi, implementare la gestione delle notifiche e della posta elettronica, configurare il server SMTP, utilizzare una libreria per l'archiviazione digitale dei file, implementare la funzione di ricerca dei documenti.


###Il risultato che si ottiene è qualitativamente buono. Per la lunghezza della trascrizione risulta essere un pò troppo verboso in quanto non riesce a sintetizzare in maniera concisa i concetti, dando però la panoramica completa di ogni punto saliente. Per quanto riguarda il secondo punto scende leggermente di qualità di output. Per quanto riguarda Andrea si riscontra un ottimo lavoro di assegnazione delle mansioni, mentre su Mario Rossi potrebbero essere riassunti in uno o due punti o meno. Su Rossana non la riconosce come client e gli assegna compiti come fosse parte del team.

#Test con l'utilizzo di un file pdf
### Qui vado a caricare un file che riprende la trascizione di un intervista alla Protezione Civile utilizzando pressochè lo stesso script. Questo perchè il codice è modulare e quindi si presta a diverse situazioni.

In [16]:
from google.colab import drive
drive.mount('/content/drive')

# Percorso completo al file (adatta se hai usato un'altra cartella)
file_path = '/content/drive/MyDrive/Project_llms/try3.pdf'

# Esempio: leggi il file (qui solo per verificare che esista)
import os
if os.path.exists(file_path):
    print('File trovato!')
else:
    print('File NON trovato!')


MessageError: Error: credential propagation was unsuccessful

In [15]:
def clean_and_tokenize_text(text):
    text_no_numbers = re.sub(r'\d+', '', text)
    words = word_tokenize(text_no_numbers, language='italian')
    stop_words = set(stopwords.words('italian'))
    cleaned_tokens = [word.lower() for word in words if word.isalpha() and word.lower() not in stop_words]
    return cleaned_tokens

url = None
pdf = file_path

transcription_text = ""

try:
    if url and not pdf:
        response = requests.get(url)
        response.raise_for_status()
        transcription_text = response.text.strip()
    elif pdf and not url:
        with pdfplumber.open(pdf) as pdf_file:
            transcription_text = "\n".join(
                page.extract_text() for page in pdf_file.pages if page.extract_text()
            ).strip()
        if not transcription_text:
            raise ValueError("Il pdf non ha testo leggibile all'interno")
    else:
        raise ValueError("Per favore inserisci solo una tra le due opzioni: url o pdf")
except Exception as e:
    transcription_text = ""
    print(f"Errore durante il caricamento: {e}")

if transcription_text:
    cleaned_tokens = clean_and_tokenize_text(transcription_text)
    cleaned_text = ' '.join(cleaned_tokens)

    llm = ChatGroq(
        api_key='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',
        model='llama3-70b-8192'
    )

    prompt_goal = PromptTemplate(
        input_variables=["transcription"],
        template="""
Sei un assistente per uffici esperto. Leggi la seguente trascrizione di un meeting e produci:
- Un riassunto diviso nei punti chiave discussi durante il meeting
- Crea per ogni persona un elenco di attività da svolgere o supervisionare, con indicazione della persona responsabile.

Trascrizione:
{transcription}

Rispondi SOLO nel formato:
---
Riassunto:
- Punto 1
- Punto 2
...

Attività:
- Persona A: attività
- Persona B: attività
...
---
"""
    )

    output = LLMChain(llm=llm, prompt=prompt_goal)

    try:
        result = output.run({"transcription": cleaned_text})
        if "---" in result:
            clean_output = result.split('---')[1].strip()
            print(clean_output)
        else:
            print(result)
    except Exception as e:
        print(f"Errore durante l'elaborazione dell'output: {e}")
else:
    print("Nessuna trascrizione valida disponibile per l'elaborazione.")


Riassunto:
- La protezione civile italiana è un sistema complesso che coinvolge diverse forze e organizzazioni, tra cui i vigili del fuoco, i volontari e le autorità regionali e nazionali.
- La gestione dei cambiamenti climatici è una delle principali sfide per la protezione civile, richiedendo l'analisi di dati e la previsione di eventi estremi.
- La comunicazione e la consapevolezza dei cittadini sono fondamentali per la prevenzione e la gestione delle emergenze.
- La tecnologia deve essere aggiornata e integrata nei processi esistenti per migliorare l'efficacia della protezione civile.
- La formazione e l'educazione civica sono essenziali per promuovere comportamenti corretti e preparare le persone alle emergenze.

Attività:
- Elisa Spalletta: analizzare i dati raccolti durante l'intervista e identificare gli ostacoli e le criticità nella gestione delle emergenze.
- Matilda Pacanowski: documentare la conversazione e identificare i punti chiave della protezione civile italiana.
- Dir

###In questo caso il modello si comporta decisamente meglio del test precedente. Il testo è molto più lungo (sono 7 pagine di intervista in cu iogni paragrafo è un pezzo di conversazione tra intervistato e intervistatore). I punti vengono riassunti in maniera ottima tralasciando cosa è superfluo. Anche il riconoscimento delle attività dei diversi enti è suddiviso correttamente.

#<font color=red>Limitazioni</font>





##Le limitazioni di questo modello sono principalmente 2. La prima che beneficiando di una velocità di esecuzione abbastanza alta potrebbe inficiare sulla qualità della comprensione del testo in sè portando ad omissioni o storpiature di alcuni temi simili o aggregare sottotemi che dovrebbero essere divisi. La seconda è collegata alla brevità che si crea nella creazione dei punti, portando a volte a delle troncature un pò brusche.


#**<font color=green>Conclusioni</font>**
#Il risultato del modello mi sembra comunque buono andando a superare tranquillamente i due test